In [1]:
import pandas as pd
from paquo.projects import QuPathProject

from pathlib import Path
from paquo.projects import QuPathProject
from paquo.images import QuPathImageType
from paquo.classes import QuPathPathClass
from shapely.geometry import Point, Polygon, MultiPoint
import numpy as np
import matplotlib.pyplot as plt
from distutils.dir_util import copy_tree


/tmp/ipykernel_24823/2887742854.py:11: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.dir_util import copy_tree


In [2]:
ml_result = pd.read_csv('../../ML_Layer_prediction/test_features_with_prediction.csv')
image_name='SLD_0000736.vsi - 20x_02'
pixel_size = 0.3460130331522824


layers_color = {"Layer 1": "#ff0000"
                , "Layer 2":"#0f00f0"
                , "Layer 3":"#0000ff"
                , "Layer 2/3":"#ffff00"
                , "Layer 4":"#f0f0f0"
                , "Layer 5":"#00f5ff"
                , "Layer 6 a":"#0ffff0"
                , "Layer 6 b":"#00ff00"
               }

ml_result = ml_result[ml_result['Image']==image_name]
ml_result

,Unnamed: 0,Image,Name,Class,Parent,ROI,Centroid X µm,Centroid Y µm,Area µm^2,Length µm,...,Smoothed: 25 µm: Cluster mean: Delaunay: Mean distance,Smoothed: 25 µm: Cluster mean: Delaunay: Median distance,Smoothed: 25 µm: Cluster mean: Delaunay: Max distance,Smoothed: 25 µm: Cluster mean: Delaunay: Min distance,Smoothed: 25 µm: Cluster mean: Delaunay: Mean triangle area,Smoothed: 25 µm: Cluster mean: Delaunay: Max triangle area,Smoothed: 25 µm: Cluster size,Smoothed: 25 µm: Nearby detection counts,image_id,rf_prediction
20563,21036,SLD_0000736.vsi - 20x_02,Layer 5,Layer 5,S1HL,Polygon,8038.3,1581.8,445.5291,109.3301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736,Layer 5
20564,21037,SLD_0000736.vsi - 20x_02,Layer 5,Layer 5,S1HL,Polygon,8745.0,1452.9,416.6770,105.8689,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736,Layer 5
20565,21038,SLD_0000736.vsi - 20x_02,Layer 5,Layer 5,S1HL,Polygon,8687.2,1574.7,386.1516,96.8733,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736,Layer 5
20566,21039,SLD_0000736.vsi - 20x_02,Layer 5,Layer 5,S1HL,Polygon,8354.9,1477.1,352.8810,92.0303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736,Layer 5
20567,21040,SLD_0000736.vsi - 20x_02,Layer 5,Layer 5,S1HL,Polygon,8221.7,1646.9,350.0055,95.4902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736,Layer 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32281,33026,SLD_0000736.vsi - 20x_02,Layer 4,Layer 4,S1HL,Polygon,8967.5,1033.8,5.1477,10.3794,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736,Layer 4
32282,33027,SLD_0000736.vsi - 20x_02,Layer 5,Layer 5,S1HL,Polygon,8528.6,1096.2,5.1471,10.3789,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736,Layer 5
32283,33028,SLD_0000736.vsi - 20x_02,Layer 5,Layer 5,S1HL,Polygon,8081.9,1539.7,4.7886,10.3801,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736,Layer 5
32284,33029,SLD_0000736.vsi - 20x_02,Layer 6 a,Layer 6 a,S1HL,Polygon,7867.7,2148.0,4.5493,8.9961,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736,Layer 6 a


In [3]:
annotations = []
#for index, layer in enumerate(["Layer 1", "Layer 2", "Layer 3", "Layer 2/3",  "Layer 4", "Layer 5", "Layer 6 a", "Layer 6 b"]):
for index, layer in enumerate(["Layer 1", "Layer 2", "Layer 3", "Layer 4", "Layer 5", "Layer 6 a", "Layer 6 b"]):
    layer_points = ml_result[ml_result['rf_prediction'] == layer][['Centroid X µm', 'Centroid Y µm']].to_numpy()
    layer_points = layer_points / pixel_size
    if layer_points.size > 0:
        
        '''
        layer_polygon = Polygon(layer_points)
        layer_polygon = Polygon(layer_polygon.convex_hull)

        x, y = layer_polygon.boundary.coords.xy
        i = index % 4
        j= int(index/4)
        ax[i][j].plot(x, y, c="red", label=layer)
    
        ax[i][j].legend()
        '''
        #layer_annotations_point = []
        '''
        for point in layer_points:
            annotation =   { '': Point(point),

            }
            layer_annotations_point.append(annotation)
        '''
        annotations.append({ layer: MultiPoint(layer_points)})
        #annotations[layer] = annotation 




In [4]:
copy_tree('./project.bck', 'project')
project_path = './project/QuPath_LayerBoundaries_GroundTruth_20220927.qpproj'
with QuPathProject(project_path, mode='r+') as qp:
    print("readed", qp.name)

    
    
    print(f'---INFO: there are {len(qp.images)}  images in this project')
    found = False
    for image in qp.images:
        if image.image_name.find(image_name) > -1:
            print(f'--- INFO: Found [{image.image_name}] ')
            for index, annotation in enumerate(annotations):
                print(f'annotation {index} / {len(annotations)}')
                
                for name, roi in annotation.items():
                    print(f'name {name}')
                    # add the annotations without a class set

                    path_class = QuPathPathClass(name='_' + name, color=layers_color[name])
                        #print(f'{name} -> {layers_color[name]}')
                    annotation = image.hierarchy.add_annotation(roi=roi, path_class=path_class)
                    #annotation.name = name
            found = True
            #print('break 1') 
            #break
        if found: 
            print('break 2')
            break

            
            

readed project/QuPath_LayerBoundaries_GroundTruth_20220927.qpproj
---INFO: there are 41  images in this project
--- INFO: Found [SLD_0000736.vsi - 20x_02] 
annotation 0 / 7
name Layer 1
15:49:56.747 [main] [DEBUG] q.lib.objects.classes.PathClass - Creating PathClass with name '_Layer 1'
annotation 1 / 7
name Layer 2
15:49:59.568 [main] [DEBUG] q.lib.objects.classes.PathClass - Creating PathClass with name '_Layer 2'
annotation 2 / 7
name Layer 3
15:49:59.569 [main] [DEBUG] q.lib.objects.classes.PathClass - Creating PathClass with name '_Layer 3'
annotation 3 / 7
name Layer 4
15:49:59.571 [main] [DEBUG] q.lib.objects.classes.PathClass - Creating PathClass with name '_Layer 4'
annotation 4 / 7
name Layer 5
15:49:59.573 [main] [DEBUG] q.lib.objects.classes.PathClass - Creating PathClass with name '_Layer 5'
annotation 5 / 7
name Layer 6 a
15:49:59.575 [main] [DEBUG] q.lib.objects.classes.PathClass - Creating PathClass with name '_Layer 6 a'
annotation 6 / 7
name Layer 6 b
15:49:59.577 [ma